---------------------------------------------
Sloučení dat z AlzaBox, Zásillkovna a GLS (finální merge pro SQL)
---------------------------------------------
Tento skript:
- Načtí 3 datasety (Alza, Zásillkovna, GLS)
- Zafixuje datové typy dle požadavků
- Sloučí je pod sebe (concat)
- Vytvoří nový jednotný ID sloupec
- Zajistí kompatibilitu s SQL databází

 Načtení dat se zafixovanými typy

In [208]:
import pandas as pd

# Datové typy dle specifikace
column_types = {
    'name': str,
    'city_code': str,
    'city': str,
    'zip_code': str,
    'street': str,
    'house_NO': str,
    'latitude': float,
    'longtitude': float
}

# Načti jednotlivé datasety
alza = pd.read_csv(r"C:\Users\Sabina\Sabi dokumenty\01 Czechitas\01 PYTHON\Python\Detektivky_vizualni_smog\sabi\data\alzaboxes_clean.csv", dtype=column_types)
zasilkovna = pd.read_csv(r"C:\Users\Sabina\Sabi dokumenty\01 Czechitas\01 PYTHON\Python\Detektivky_vizualni_smog\sabi\data\zasilkovna_clean.csv", dtype=column_types)
gls = pd.read_csv(r"C:\Users\Sabina\Sabi dokumenty\01 Czechitas\01 PYTHON\Python\Detektivky_vizualni_smog\sabi\data\ss_gls_clean_dataset.csv", dtype=column_types, encoding='utf-8')

# Očištění mezer v názvech sloupců
alza.columns = alza.columns.str.strip()
zasilkovna.columns = zasilkovna.columns.str.strip()
gls.columns = gls.columns.str.strip()

# Přetypování sloupců, včetně konverze prázdných hodnot na NaN
for df in [alza, zasilkovna]:
    df['latitude'] = pd.to_numeric(df['latitude'], errors='coerce')
    df['longtitude'] = pd.to_numeric(df['longtitude'], errors='coerce')

# Ujistěme se o typech i v GLS (pokud by se při exportu ztratily)
gls = gls.astype(column_types)

# Označení zdroje
alza['source'] = 'alzabox'
zasilkovna['source'] = 'zasilkovna'
gls['source'] = 'gls'

2. Spojení tabulek pod sebe (bez ztráty dat)

In [209]:
df = pd.concat([gls, alza, zasilkovna], ignore_index=True)

3. Vytvoření nového ID (od 1)

In [210]:
df['id'] = range(1, len(df) + 1)
df['id'] = df['id'].astype(int)

4. Uspořádání sloupců podle specifikace

In [211]:
ordered_cols = ['id', 'name', 'city_code', 'city', 'zip_code', 'street', 'house_NO', 'latitude', 'longtitude', 'source']
df = df[ordered_cols]

5. Kontrola datových typů 

In [212]:
print(df.dtypes)

id              int64
name           object
city_code      object
city           object
zip_code       object
street         object
house_NO       object
latitude      float64
longtitude    float64
source         object
dtype: object


6. Uložení finálního datasetu (např. pro SQL import)

In [213]:
df.to_csv(r"C:\Users\Sabina\Sabi dokumenty\01 Czechitas\01 PYTHON\Python\Detektivky_vizualni_smog\sabi\data\merged_boxes_final.csv", index=False, encoding='utf-8')

1. Dodatečné úpravy datasetu

In [214]:
import pandas as pd

column_types = {
    'name': str,
    'city_code': str,
    'city': str,
    'zip_code': str,
    'street': str,
    'house_NO': str,
    'latitude': float,
    'longtitude': float
}
data = pd.read_csv(r"C:\Users\Sabina\Sabi dokumenty\01 Czechitas\01 PYTHON\Python\Detektivky_vizualni_smog\sabi\data\merged_boxes_final.csv")

df = pd.DataFrame(data)

In [215]:
df.columns

Index(['id', 'name', 'city_code', 'city', 'zip_code', 'street', 'house_NO',
       'latitude', 'longtitude', 'source'],
      dtype='object')

In [216]:
chybi_city = df[df['city'].isna()]
chybi_city

,id,name,city_code,city,zip_code,street,house_NO,latitude,longtitude,source
3390,3391,Alza Box,533025,NaN,27361.0,Lesní,383,50.106031,14.069672,alzabox


In [217]:
doplnit_city = {
    3390: 'Velká Dobrá'
    # ...
}

# Přiřazení hodnot do sloupce house_no
for idx, mesto in doplnit_city.items():
    df.loc[idx, 'city'] = mesto

# Volitelná kontrola
df.loc[doplnit_city.keys(), ['city_code', 'city']]

,city_code,city
3390,533025,Velká Dobrá


In [218]:
chybi_city = df[df['city'].isna()]
chybi_city

,id,name,city_code,city,zip_code,street,house_NO,latitude,longtitude,source


V případě Alzaboxů chybí 150 záznamů ulice, to se musí opravit
Oprava je škaredá, ale jinak to teď už nejde.

In [219]:
chybi_street = df[df['street'].isna()]
chybi_street

,id,name,city_code,city,zip_code,street,house_NO,latitude,longtitude,source
1010,1011,Alza Box,531057,Beroun,26601.0,NaN,NaN,49.956792,14.062212,alzabox
1033,1034,Alza Box,584037,Tvarožná,66405.0,NaN,NaN,49.184367,16.758946,alzabox
1034,1035,Alza Box,577839,Borová,56982.0,NaN,NaN,49.746354,16.154598,alzabox
1044,1045,Alza Box,570788,Bradlec,29306.0,NaN,NaN,50.442856,14.899646,alzabox
1045,1046,Alza Box,538094,Brandýs nad Labem-Stará Boleslav,25001.0,NaN,NaN,50.196479,14.687264,alzabox
...,...,...,...,...,...,...,...,...,...,...
3421,3422,Alza Box,541630,Vsetín,75501.0,NaN,NaN,49.331542,17.986446,alzabox
3452,3453,Alza Box,539066,Zeleneč,25091.0,NaN,NaN,50.130371,14.664706,alzabox
3475,3476,Alza Box,593711,Znojmo,66902.0,NaN,NaN,48.868348,16.033585,alzabox
3500,3501,Alza Box,563871,Železný Brod,46822.0,NaN,NaN,50.641223,15.255739,alzabox


In [220]:
# Slovník: id → název ulice
doplnit_ulici = {
    1011: "(OMV u Kauflandu)",
    1034: "Tvarožná - D1 (OMV)",
    1035: "Borová u Poličky (G.A.S. Petroleum)",
    1045: "Sportovní",
    1046: "(Ceiba)",
    1056: "Sportovců (U Hřiště)",
    1080: "Svratecká",
    1081: "Za Kovárnou",
    1083: "Hrnčířská",
    1089: "Franzova",
    1130: "Stará osada",
    1183: "Sportovní (Parkoviště)",
    1219: "Nad Jezem",
    1268: "Husova tř. (Kaufland)",
    1282: "Retail park Okružní",
    1439: "Za Zámkem (Parkoviště za Coopem)",
    1447: "Hodonín (OMV)",
    1448: "Tovární Shop zone",
    1504: "Labská louka (Penny)",
    1512: "Sokolská (OMV)",
    1534: "Okružní (Automyčka)",
    1537: "Brněnská (Kaufland)",
    1595: "Ivanovice na Hané (Penny)",
    1615: "Jaroměřice nad Rokytnou (Bus nádraží)",
    1629: "Jeseník (Penny)",
    1659: "Claudiusova (VŠE)",
    1663: "Hlavní (Super Zoo)",
    1719: "Stará",
    1721: "Arménská (Parkoviště)",
    1722: "Pařížská (Parkoviště)",
    1734: "Kročehlavská (OMV)",
    1754: "Autobusová zastávka směr Praha",
    1761: "Březinova",
    1771: "Buzulucká (Parkoviště)",
    1786: "Kostomlaty nad Labem (Coop)",
    1799: "Kralupy nad Vltavou - Mikovice (Medos Ina)",
    1827: "Ke Dvoru (Parkoviště)",
    1865: "Letná (Autokemp)",
    1867: "Tržní náměstí",
    1872: "Divišova (Mocca)",
    1874: "1. máje (Parkoviště hotelu)",
    1888: "Máchova",
    1902: "Autobusová zastávka",
    1909: "Na Výsluní (Parkoviště)",
    1919: "Litovel (Penny)",
    1945: "Lovosice (Kaufland)",
    1957: "Lutín (Billa)",
    1961: "Na Mlíčníku",
    1965: "Plzeňská (OMV)",
    1981: "Městec Králové (Penny)",
    2000: "Hrusická",
    2006: "Zalužanská",
    2007: "Bezděčínská",
    2024: "5. května (Maso, uzeniny)",
    2032: "Moravská Nová Ves (Chemis engine)",
    2034: "Svitavská (Kaufland)",
    2086: "Zagarolská (STAB)",
    2113: "Na Bořetíně",
    2130: "Autobusové nádraží",
    2136: "Dělnická",
    2152: "Peškova (Parkoviště)",
    2195: "Oslavany (Autobusové nádraží)",
    2206: "Mannesmannova (Myčka)",
    2238: "Přívoz (OC Koksovna LIDL)",
    2247: "Dr. Lukášové křižovatka Holasova",
    2273: "Kolínská (Autobusová zastávka)",
    2275: "Jana Autengrubera (Penny)",
    2293: "Chrudimská (Eurobit)",
    2296: "Hradecká (OMV)",
    2302: "Tř. Jana Švermy (Billa)",
    2322: "Městský Úřad",
    2324: "Zahradní",
    2328: "Slovany-UŠkolky",
    2333: "NaBelánce(Norma)",
    2342: "Bolevec-Kaznějovská",
    2351: "Skvrňany-NadeMží",
    2363: "UVelkéhorybníka(Parkoviště)",
    2379: "Žirovnická (Penny)",
    2390: "Znojemská (Retail Park)",
    2414: "Záběhlice - V Korytech (KRAFT)",
    2415: "Dolní Měcholupy - Dolnoměcholupská",
    2429: "Strašnice - V Korytech (MOL)",
    2438: "V Olšinách (Strašnická podchod metra)",
    2446: "Strašnice - Úvalská (Automyčka)",
    2504: "Lhotecká (OMV)",
    2533: "Dolní Počernice (Camping Sokol)",
    2553: "Horní Měcholupy (HAPPI Milánská)",
    2554: "Horní Měcholupy - Janovská",
    2578: "Velká Chuchle - Strakonická (OMV)",
    2593: "Slánská (OMV)",
    2597: "Letňany - Veselská (Odien Real)",
    2599: "Čakovice - Třeboradice - Schollerov",
    2618: "Kbely - Mladoboleslavská (OMV)",
    2624: "Nusle - Svatoplukova (MOL)",
    2648: "Újezd - Josefa Bíbrdlíka",
    2650: "Újezd nad Lesy (OMV)",
    2651: "Kolovraty - Měsíčková (Parkoviště)",
    2654: "Čestlice - K Dálnici (Kaufland)",
    2701: "Nusle (Autobusová zastávka - Děkanka)",
    2705: "Michle - (Budova Filadelfie)",
    2712: "Krč - Olbrachtova (MOL)",
    2713: "Nusle (Tramvajová zastávka Otakarova)",
    2746: "Chodovská (OMV)",
    2750: "Slivenec (Kaufland)",
    2753: "Smíchov - Vackové (Smíchov City)",
    2761: "Smíchov - Peroutkova (Autobazar)",
    2762: "Smíchov - Strakonická (MOL)",
    2777: "Plzeňská (OMV směr z centra)",
    2792: "Břevnov - Ankarská (Koleje)",
    2795: "Bubeneč - Maďarská/Juárezova",
    2796: "Břevnov - U Větrníku",
    2831: "Holešovice - Železničářů",
    2832: "Holešovice - Partyzánská (Port7)",
    2861: "Čimice - Na Průhonu (Tenisová škola)",
    2879: "Libeň - Nad Okrouhlíkem",
    2882: "Libeň (Myčka Vosmíkových)",
    2895: "Vysočany - Kbelská (MOL)",
    2896: "Hloubětín - Nademlejnská",
    2903: "Vysočany - Kolbenova (Metro)",
    2905: "Na Černé Strouze (Výhledy Rokytka)",
    2941: "Zelenohorská (NORMA)",
    2947: "9.května (Myčka)",
    2965: "Seifertova (U Albertu)",
    2967: "Evropská (OMV)",
    2990: "Klostermannova",
    3014: "Rožnov pod Radhoštěm (OMV)",
    3019: "Parkoviště ul. Na Plácku",
    3030: "Tyrše a Fügnera",
    3037: "Tylova (Městská tržnice)",
    3038: "Čerpací stanice KM Prona",
    3085: "Komenského (Parkoviště)",
    3096: "Stehelčeves (Retail Park)",
    3106: "Strakonice (OMV)",
    3138: "Svoboda nad Úpou (Eurobit)",
    3161: "Skatepark Štoky",
    3185: "Tachov (Tesco)",
    3192: "Temelín (Elektrárna Temelín)",
    3211: "Terezín - Retail Park (Pepco)",
    3219: "Tlumačov (ČS PRIM)",
    3263: "Tuchoměřice - Ke Kopanině (Penny)",
    3283: "Uherský Brod (Automyčka Wash Me)",
    3338: "Za Radnicí (Autobusové nádraží)",
    3361: "Karlov",
    3375: "Sokolská",
    3386: "Tučapy u Soběslavi (G.A.S. Petroleum)",
    3422: "Rokytnice (BKF Carwash)",
    3453: "Husova (Vlakové nádraží)",
    3476: "Pražská (OMV)",
    3501: "Masarykova",
    3502: "Židlochovice (Lidl)"
}

# Přiřazení do sloupce 'street'
for item, ulice in doplnit_ulici.items():
    df.loc[df['id'] == item, 'street'] = ulice

# Volitelná kontrola
df.loc[doplnit_ulici.keys(), ['city_code', 'street']]

,city_code,street
1011,567451,Teplická
1034,577839,Borová u Poličky (G.A.S. Petroleum)
1035,544281,Žižkovo náměstí
1045,538094,(Ceiba)
1046,538094,P. Jilemnického
...,...,...
3422,541630,Jasenická
3453,539881,U Jezera
3476,593711,Jarošova
3501,584282,Židlochovice (Lidl)


In [221]:
df['zip_code'] = df['zip_code'].astype(str).str.replace(r'\.0$', '', regex=True)

In [222]:
df.head()

,id,name,city_code,city,zip_code,street,house_NO,latitude,longtitude,source
0,1,GLS Parcel Box,586846,Jihlava,58601,S. K. Neumanna,590/22,49.4048,15.5570,gls
1,2,GLS Parcel Box,597007,Velké Meziříčí,59401,Sokolovská,147/25,49.3517,16.0196,gls
2,3,GLS Parcel Box,545058,Valašské Meziříčí,75701,Králova,1256,49.4693,17.9711,gls
3,4,GLS Parcel Box,554782,Praha,15600,Pod Třešňovkou,234,49.9708,14.3832,gls
4,5,GLS Parcel Box,511382,Přerov,75002,Svisle,1100/9,49.4503,17.4594,gls


In [223]:
df = df.drop(columns=['source'])

In [224]:
df['house_NO'] = df['house_NO'].astype(str)
df['zip_code'] = df['zip_code'].astype(str)
df['city_code'] = df['city_code'].astype(str)
df['street'] = df['street'].astype(str)
df['latitude'] = df['latitude'].astype(float)
df['longtitude'] = df['longtitude'].astype(float)

In [227]:
df.to_csv(
    r'C:\Users\Sabina\Sabi dokumenty\01 Czechitas\01 PYTHON\Python\Detektivky_vizualni_smog\sabi\data\ss_merged_gls_alza_zbox_dataset.csv', 
    sep=";",                  # 1. oddělovač středník
    index=False,              # bez indexu
    quoting=1,                # 3. obalit všechny stringy do uvozovek
    quotechar='"',            # uvozovky
    encoding="utf-8"          # doporučené kódování
    )